In [9]:
import gc
import torch
from fastai.vision import *
from fastai.widgets import *
from fastai.callbacks import * 
import shutil
import fastai
print(f'fastai: {fastai.__version__}')
print(f'cuda: {torch.cuda.is_available()}')

fastai: 1.0.52
cuda: False


In [10]:
Config.DEFAULT_CONFIG = {
        'data_path': './../data/MURA-v1.1/',
        'model_path': './models/'
    }

Config.create('/tmp/myconfig.yml')
Config.DEFAULT_CONFIG_PATH = '/tmp/myconfig.yml'

path = Config.data_path()

In [103]:
size = 320
bs = 32

np.random.seed(24)

def create_from_ll(lls:LabelLists, bs:int=64, val_bs:int=None, ds_tfms:Optional[TfmList]=None,
                num_workers:int=defaults.cpus, dl_tfms:Optional[Collection[Callable]]=None, device:torch.device=None,
                test:Optional[PathOrStr]=None, collate_fn:Callable=data_collate, size:int=None, no_check:bool=False,
                resize_method:ResizeMethod=None, mult:int=None, padding_mode:str='reflection',
                mode:str='bilinear', tfm_y:bool=False)->'ImageDataBunch':
    "Create an `ImageDataBunch` from `LabelLists` `lls` with potential `ds_tfms`."
    lls = lls.transform(tfms=get_transforms(do_flip=True, max_rotate=30, max_warp=0.0, p_lighting=0, xtra_tfms=[]), size=size, resize_method=resize_method, mult=mult, padding_mode=padding_mode,
                        mode=mode, tfm_y=tfm_y)
    
    return lls.databunch(bs=bs, val_bs=bs, dl_tfms=dl_tfms, num_workers=num_workers, collate_fn=collate_fn,
                         device=device, no_check=True)

src = (ImageList.from_df(df=pd.read_csv('all_WRIST.csv'), path='../data/MURA-v1.1/data2/', folder=None, suffix='', cols=0)
                .split_from_df("is_valid")
                .label_from_df(label_delim=None, cols=1))
data = create_from_ll(src, size = size, bs = bs).normalize(imagenet_stats)

kappa = KappaScore()
kappa.weights = "quadratic"



In [104]:
learner = cnn_learner(data, models.vgg19_bn, metrics=[error_rate, accuracy, kappa], wd=0.1, model_dir="./models/").to_fp32()
# #learner.unfreeze()
# learner.fit_one_cycle(5,max_lr=1e-3, callbacks=[ShowGraph(learner) ,SaveModelCallback(learner)])

learner.load('vgg19bn-WRISTfullval')

Learner(data=ImageDataBunch;

Train: LabelList (9752 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
0,0,0,0,0
Path: ../data/MURA-v1.1/data2;

Valid: LabelList (659 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
0,0,0,0,0
Path: ../data/MURA-v1.1/data2;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, k

In [105]:
# interp = ClassificationInterpretation.from_learner(learner)

# losses,idxs = interp.top_losses()

# len(data.valid_ds)==len(losses)==len(idxs)

In [106]:
# interp.plot_confusion_matrix()

In [107]:
# interp.plot_top_losses(30, figsize=(30,50))

In [108]:
val_preds,val_targets = learner.get_preds() 

In [109]:
print(float(val_preds[0][0]))

0.9722619652748108


In [110]:
from os import listdir
from os.path import isfile, join
files = [f for f in listdir('../data/MURA-v1.1/data2/valid/0/') if isfile(join('../data/MURA-v1.1/data2/valid/0/', f)) and "WRIST" in f]
files += [f for f in listdir('../data/MURA-v1.1/data2/valid/1/') if isfile(join('../data/MURA-v1.1/data2/valid/1/', f)) and "WRIST" in f]
files[0]

'WRIST_patient11347_study2_negative_image1.png'

In [111]:
studies = []
studies_labels = []
for file in files:
    study = file[:-11]
    if study not in studies:
        studies.append(study)
        if 'negative' in study:
            studies_labels.append(0)
        else:
            studies_labels.append(1)


In [112]:
# learner.get_preds('../data/MURA-v1.1/data2/valid/0/ELBOW_patient11860_study1_negative_image1.png')

# img = open_image('../data/MURA-v1.1/data2/valid/0/ELBOW_patient11860_study1_negative_image1.png')
# img
# learner.predict(img)

print(len(val_preds))
print(len(files))

659
659


In [113]:
study_actual = dict()
for study in studies:
    if 'negative' in study:
        study_actual[study] = 0
    else:
        study_actual[study] = 1
    

In [114]:
study_dict = dict()
for study in studies:
    if study not in study_dict:
        study_dict[study] = []

In [115]:
file_count = 0
for file in files:
    study = file[:-11]
    study_dict[study].append(float(val_preds[file_count][1]))
    file_count += 1

In [116]:
study_preds = dict()
for study in study_dict:
    combined_pr = sum(study_dict[study])/len(study_dict[study])
    if combined_pr >= 0.5:
        study_preds[study] = 1
    else:
        study_preds[study] = 0

In [117]:
print(len(study_actual))
print(len(study_preds))

237
237


In [118]:
please = 0
for i in study_actual:
    if study_actual[i] == study_preds[i]:
        please += 1

In [119]:
please/len(study_preds)

0.8860759493670886

In [120]:
#kappa
actual_list = []
pred_list = []
for i in study_actual:
    actual_list.append(study_actual[i])
    pred_list.append(study_preds[i])



In [121]:
p_o = 0
p_e = 0

In [122]:
agreed = 0
for i in range(len(actual_list)):
    if actual_list[i] == pred_list[i]:
        agreed += 1
p_o = agreed/len(actual_list)
p_o

0.8860759493670886

In [123]:
actual_yes = sum(actual_list)/len(actual_list)
pred_yes = sum(pred_list)/len(pred_list)

actual_no = 1-sum(actual_list)/len(actual_list)
pred_no = 1-sum(pred_list)/len(pred_list)

p_e = actual_yes*pred_yes + actual_no*pred_no
p_e

0.527942459363706

In [124]:
kappa = (p_o-p_e)/(1-p_e)

In [125]:
kappa

0.7586649066566094